In [1]:
from IPython.display import display_html

import logging
import warnings
import re
import os
import numpy as np
import pandas as pd
import pickle
import sklearn.metrics
import pickle
import requests
import collections
import matplotlib.pyplot as plt
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, MACCSkeys
from datetime import date
from typing import Literal, Optional, Union, List, Dict, Tuple, Any, Callable
from collections import defaultdict

from IPython.display import display_html
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'

import collections
import itertools
import re
import gc
import math
import numpy as np
import pandas as pd
import pickle
import requests as r
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import random
import copy
import os

import typing
from typing import Mapping, Literal, Callable, List, ClassVar, Any, Tuple, Type

from uuid import uuid4
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs, MACCSkeys, Draw
from rdkit.Chem.Draw import IPythonConsole
from datetime import date
from scipy.sparse import csr_matrix, vstack
from tqdm import tqdm

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils import resample, class_weight

import joblib

pd.set_option('display.max_columns', 1000, 'display.width', 2000, 'display.max_colwidth', 100)

In [2]:
data_dir = os.path.join(os.getcwd(), '..', 'data')
src_dir = os.path.join(os.getcwd(), '..', 'src')
fig_dir = os.path.join(data_dir, 'figures')
checkpoint_dir = os.path.join(os.getcwd(), '..', 'checkpoints')
dirs_to_make = [
    data_dir,
    os.path.join(data_dir, 'raw'),
    os.path.join(data_dir, 'processed'),
    os.path.join(data_dir, 'train'),
    os.path.join(data_dir, 'val'),
    os.path.join(data_dir, 'test'),
    src_dir,
    fig_dir,
    checkpoint_dir,
]
for d in dirs_to_make:
    if not os.path.exists(d):
        os.makedirs(d)

In [7]:
ssl_df_path = os.path.join(data_dir, 'processed', 'protac_db_ssl.csv')
train_df_path = os.path.join(data_dir, 'train', 'train_bin_upsampled.csv')
val_df_path = os.path.join(data_dir, 'val', 'val_bin.csv')
test_df_path = os.path.join(data_dir, 'test', 'test_bin.csv')
cols_to_keep = [
    'Smiles',
    'Smiles_nostereo',
    'DC50',
    'pDC50',
    'Dmax',
    'poi_gene_id',
    'poi_seq',
    'cell_type',
    'e3_ligase',
    'active',
]
# Assign train/val datasets for use in dataloaders
train_df = pd.read_csv(train_df_path).reset_index(drop=True)
val_df = pd.read_csv(val_df_path).reset_index(drop=True)
test_df = pd.read_csv(test_df_path).reset_index(drop=True)
ssl_df = pd.read_csv(ssl_df_path).reset_index(drop=True)
train_df = train_df[cols_to_keep]
val_df = val_df[cols_to_keep]
test_df = test_df[cols_to_keep]
ssl_df = ssl_df[cols_to_keep]

protac_df = pd.concat([train_df, val_df, ssl_df]).reset_index(drop=True)
protac_df = protac_df[cols_to_keep]
protac_df.head()

,Smiles,Smiles_nostereo,DC50,pDC50,Dmax,poi_gene_id,poi_seq,cell_type,e3_ligase,active
0,Cc1ncsc1-c1ccc(CNC(=O)[C@@H]2C[C@@H](O)CN2C(=O)[C@@H](NC(=O)CCCCCCCCCCNC(=O)c2cc3c(cc2CS(C)(=O)=...,C(CCCCCNC(=O)c1cc2c(-c3cn(C)c(=O)c4c3c(c[nH]4)CN2c2ncc(F)cc2F)cc1CS(=O)(C)=O)CCCCC(=O)NC(C(N1CC(...,3.400000e-09,8.468521,0.980000,BRD4,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPVDAVKLNLPD...,PC3-S1,VHL,True
1,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)COCCOCCOCCN2CCN(c3ccc(Nc4ncc5c(C)c(C(C)=O)c(=O)n(C6C...,C(=O)(C(C(C)(C)C)NC(=O)C(NC)C)N1CC(NC(COCCOCCOCCN2CCN(c3ccc(Nc4ncc5c(C)c(C(=O)C)c(=O)n(C6CCCC6)c...,1.600000e-08,7.795880,0.871440,CDK6,MEKDGLCRADQQYECVAEIGEGAYGKVFKARDLKNGGRFVALKRVRVQTGEEGMPLSTIREVAVLRHLETFEHPNVVRLFDVCTVSRTDRETKLTL...,JURKAT,IAP,True
2,O=C(CCCCCCC(=O)N/N=C/c1ccc(OCCCC#Cc2cccc3c2CN(C2CCC(=O)NC2=O)C3=O)cc1)NO,C(Oc1ccc(C=NNC(CCCCCCC(NO)=O)=O)cc1)CCC#Cc1cccc2c1CN(C1CCC(=O)NC1=O)C2=O,1.940000e-09,8.712198,0.896614,HDAC6,MTSTGQDSTTTRQRRSRQNPQSPPQDSSVTSKRNIKKGAVPRSIPNLAEVKKKGKMKKLGQAMEEDLIVGLQGMDLNLEAEALAGTGLVLDEQLNE...,MM1S,CRBN,True
3,Cc1ncsc1-c1ccc(CNC(=O)[C@@H]2C[C@@H](O)CN2C(=O)[C@@H](NC(=O)CCc2cc(N)cc(CCOCCOCC#Cc3ccc(C4=N[C@@...,CC(C)(C(C(N1C(C(=O)NCc2ccc(-c3c(C)ncs3)cc2)CC(O)C1)=O)NC(CCc1cc(N)cc(CCOCCOCC#Cc2ccc(C3=NC(Cc4nc...,8.700000e-08,7.060481,1.000000,BRD4,MSAESGPGTRLRNLPVMGDGLETSQMSTTQAQAQPQPANAASTNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPVDAVKLNLPD...,PC3-S1,VHL,True
4,Nc1ncnc2c1c(-c1ccc(Oc3ccccc3)cc1)nn2[C@@H]1CCCN(C(=O)/C=C/COCCOCCOCCOCCOCCOc2cccc3c2C(=O)N(C2CCC...,C(Oc1cccc2c1C(=O)N(C1CCC(=O)NC1=O)C2=O)COCCOCCOCCOCCOCC=CC(N1CC(n2c3c(c(N)ncn3)c(-c3ccc(Oc4ccccc...,8.600000e-09,8.065502,0.910000,BTK,MAAVILESIFLKRSQQKKKTSPLNFKKRLFLLTVHKLSYYEYDFERGRRGSKKGSIDVEKITCVETVVPEKNPPPERQIPRRGEESSEMEQISIIE...,MINO,CRBN,True


## POI Sequence Encoding

#### POI Sequence to $N_{grams}$

Count-vectorize the POI amino acid sequence.

(Not ideal and very simple, but it's a start)

In [11]:
ngram_min_range = 2 # Orginal: 3
ngram_max_range = 2 # Orginal: 3
# poi_vectorizer = CountVectorizer(analyzer='char', ngram_range=(ngram_min_range, ngram_max_range))
# X = poi_vectorizer.fit_transform(protac_df['poi_seq'].tolist())
# rec_n_grams_df = pd.DataFrame(X.toarray(), columns=list(s.replace(' ', '') for s in poi_vectorizer.get_feature_names_out()))
# print(f'POI embedding size: {rec_n_grams_df.shape[-1]}')

protac_df['poi_seq'] = protac_df['poi_seq'].fillna('')

# Load the pre-trained countvectorizer if it exists, otherwise train it
poi_encoder_filepath = os.path.join(checkpoint_dir, 'poi_encoder.joblib')
if os.path.exists(poi_encoder_filepath):
    print('Loading pre-trained POI vectorizer...')
    poi_encoder = joblib.load(poi_encoder_filepath)
else:
    print('Training POI vectorizer...')
    poi_encoder = CountVectorizer(analyzer='char', ngram_range=(ngram_min_range, ngram_max_range))
    X = poi_encoder.fit_transform(protac_df['poi_seq'].tolist())
    rec_n_grams_df = pd.DataFrame(X.toarray(), columns=list(s.replace(' ', '') for s in poi_encoder.get_feature_names_out()))
    print(f'POI embedding size: {rec_n_grams_df.shape[-1]}')
    joblib.dump(poi_encoder, poi_encoder_filepath)
print('Done!')

Training POI vectorizer...
POI embedding size: 403
Done!


#### POI Gene Ordinal Encoding

Add the "Unknown" class to the POI genes.

Since genes ultimately encode proteins, we can use the gene ID as a categorical feature to include information about the POIs.

(The information loss is considerable, since the gene ID is not that informative compared to the entire amino acid sequence)

In [ ]:
poi_gene_enc = preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value',
                                            unknown_value=-1,
                                            encoded_missing_value=-1)
poi_gene_id = protac_df['poi_gene_id'].to_numpy().reshape(-1, 1)
poi_gene_enc.fit(poi_gene_id)

## E3 Ligase and Cell Type Ordinal Encoding

Notice that the "other E3" have been dropped during the previous steps, leading to only 5 possibilities left.

In [13]:
# Load the pre-trained ordinal encoder if it exists, otherwise train it
e3_encoder_filepath = os.path.join(checkpoint_dir, 'e3_ligase_encoder.joblib')
if os.path.exists(e3_encoder_filepath) and False:
    print('Loading pre-trained POI vectorizer...')
    e3_encoder = joblib.load(e3_encoder_filepath)
else:
    print('Training E3 encoder...')
    e3_encoder = preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value',
                                              unknown_value=-1,
                                              encoded_missing_value=-1)
    e3_ligase = protac_df['e3_ligase'].to_numpy().reshape(-1, 1)
    e3_encoder.fit(e3_ligase)
    joblib.dump(e3_encoder, e3_encoder_filepath)
print('Done!')

Training E3 encoder...
Done!


In [12]:
# Load the pre-trained ordinal encoder if it exists, otherwise train it
cell_encoder_filepath = os.path.join(checkpoint_dir, 'cell_type_encoder.joblib')
if os.path.exists(cell_encoder_filepath):
    print('Loading pre-trained POI vectorizer...')
    cell_encoder = joblib.load(cell_encoder_filepath)
else:
    print('Training E3 encoder...')
    cell_encoder = preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1,
                                                encoded_missing_value=-1)
    cell_type = protac_df['cell_type'].to_numpy().reshape(-1, 1)
    cell_encoder.fit(cell_type)
    joblib.dump(cell_encoder, cell_encoder_filepath)
print('Done!')

Training E3 encoder...
Done!
